In [1]:
import os
import sys
import re

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/spark/third-party-jars/spark-streaming-kafka-0-10_2.12-3.1.1.jar;/usr/local/spark/third-party-jars/spark-sql-kafka-0-10_2.12-3.1.1.jar pyspark-shell'

# Configure the environment
# if 'SPARK_HOME' not in os.environ:
# os.environ['SPARK_HOME'] = '/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7'
# Create a variable for our root path
# SPARK_HOME = os.environ['SPARK_HOME']
# print(SPARK_HOME)
#Set python location 
# sys.path.append("/usr/local/lib/python3.7/site-packages")
#Set python location 
# sys.path.append("/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7/python")
# sys.path.append("/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip")


from pyspark.sql import SparkSession, SQLContext
from pyspark import  SparkContext, SparkConf
from pyspark.streaming import StreamingContext

# Spark session & context
conf = SparkConf()
conf.setMaster("local").setAppName("hello-world")
conf.set("spark.executor.memory", "1g")
conf.set("spark.executor.cores", "1")
conf.set("spark.cores.max", "4")
conf.set("spark.driver.memory",'1g')
conf.set("spark.driver.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.executor.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.sql.caseSensitive", "true")
conf.set("spark.ui.port", "4040")

sc = SparkContext(conf=conf)
spark = SparkSession(sc)
sqlContext = SQLContext(sc)
ssc = StreamingContext(sc, 1)

In [ ]:
from pyspark.sql import functions
from pyspark.sql.types import *

# Read Kafka

In [43]:
# Subscribe to 1 topic defaults to the earliest and latest offsets
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe", "buses-location") \
  .load()
df = df\
    .withColumn('key', df.key.cast(StringType()))\
    .withColumn('value', df.value.cast(StringType()))\

df.show(1)

+---+--------------------+--------------+---------+------+--------------------+-------------+
|key|               value|         topic|partition|offset|           timestamp|timestampType|
+---+--------------------+--------------+---------+------+--------------------+-------------+
|180|{"route_id": "180...|buses-location|        5|     0|2021-05-20 17:19:...|            0|
+---+--------------------+--------------+---------+------+--------------------+-------------+
only showing top 1 row



In [16]:
schema = StructType([
    StructField("route_id", StringType(), True),
    StructField("id", StringType(), True),
    StructField("run_id", StringType(), True),
    StructField("predictable", BooleanType(), True),
    StructField("seconds_since_report", LongType(), True),
    StructField("heading", DoubleType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

In [44]:
df_1 = df\
    .select(functions.from_json("value", schema).alias("bus_location"))\
    .select("bus_location.*")

df_1.show(1, False)

+--------+----+---------+-----------+--------------------+-------+---------+-----------+
|route_id|id  |run_id   |predictable|seconds_since_report|heading|latitude |longitude  |
+--------+----+---------+-----------+--------------------+-------+---------+-----------+
|180     |5859|180_177_1|true       |140                 |195.0  |34.120403|-118.271271|
+--------+----+---------+-----------+--------------------+-------+---------+-----------+
only showing top 1 row

